In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import requests
from bs4 import BeautifulSoup
import time

In [2]:
#grab list of recipe titles and links 
allrecipes_db = pd.read_csv('allrecipes_db_all.csv')
print('number of recipes:',len(allrecipes_db))
allrecipes_nutrdb = pd.concat([pd.read_csv('allrecipes_nutrdb.csv'),pd.read_csv('allrecipes_nutrdb2.csv'),
                              pd.read_csv('allrecipes_nutrdb3.csv')])
print('number of recipes with nutrfacts:',len(allrecipes_nutrdb))

number of recipes: 6899
number of recipes with nutrfacts: 6497


In [55]:
allrecipes_db.head()

,Unnamed: 0,ratings,numreviews,links,recipename
0,0,4.418605,63,https://www.allrecipes.com/recipe/223175/grill...,grilled chicken with peach sauce
1,1,4.738095,2,https://www.allrecipes.com/recipe/263611/chica...,chicago deep dish pizza
2,2,4.440678,7,https://www.allrecipes.com/recipe/261124/chimi...,chimichurri baked chicken
3,3,4.661208,1,https://www.allrecipes.com/recipe/261325/roast...,roasted eggplant pastitsio
4,4,4.718750,6,https://www.allrecipes.com/recipe/263799/phill...,philly cheese steak stuffed bell peppers


In [67]:
# #loop each recipe on allrecipes_db and extract nutrfacts
nutrdict = {}
errorID = []
nodataID = []
cnt = 1
for num in range(3000,4000):
    url = allrecipes_db.iloc[num].links+'fullrecipenutrition'
    r = requests.get(url)
    if r.status_code == requests.codes.ok:
        soup = BeautifulSoup(r.text, 'html.parser')
        try:
            calories = [x for x in soup.find('div',attrs={'class':'nutrition-top light-underline'}).stripped_strings]
            calories = int(calories[-1])
            nutrientnames = soup.find_all('span',attrs={'class':'nutrient-name'})
            nutrientnames = [x.contents[0].replace(': ','') for x in nutrientnames]
            nutrientvals = soup.find_all('span',attrs={'class':'nutrient-value'})
            nutrientvals = [x.contents[0].replace(': ','') for x in nutrientvals]
            nutrientvals.append(calories)
            nutrientnames.append('Calories')
            nutrdict[allrecipes_db.iloc[num].recipename] = dict(x for x in zip(nutrientnames,nutrientvals))
        except: 
            nodataID.append(num)
    else:
        errorID.append(num)
        print('bad request on iter:',num)
    
    if num % 100 == 0:
        print('finished with row:',num,'/',str(len(allrecipes_db)))

    if num == len(allrecipes_db): 
        print('done')
        
    time.sleep(1.1)

allrecipes_nutrdb = pd.DataFrame.from_dict(nutrdict,orient='index')
allrecipes_nutrdb.to_csv('allrecipes_nutrdb_all_3000to4000.csv')
print(len(allrecipes_nutrdb))
allrecipes_nutrdb.tail()

finished with row: 3000 / 6899
finished with row: 3100 / 6899
finished with row: 3200 / 6899
finished with row: 3300 / 6899
finished with row: 3400 / 6899
finished with row: 3500 / 6899
finished with row: 3600 / 6899
finished with row: 3700 / 6899
finished with row: 3800 / 6899
finished with row: 3900 / 6899
911


,Total Fat,Saturated Fat,Cholesterol,Sodium,Potassium,Total Carbohydrates,Dietary Fiber,Protein,Sugars,Vitamin A,Vitamin C,Calcium,Iron,Thiamin,Niacin,Vitamin B6,Magnesium,Folate,Calories
yummy pork noodle casserole,32.9g,17.0g,142mg,1575mg,615mg,32.9g,2g,38.2g,4g,1998IU,9mg,344mg,4mg,1mg,15mg,1mg,60mg,73mcg,580
zesty chicken and potatoes,9.9g,3.0g,59mg,558mg,725mg,41.1g,4.4g,18.5g,7g,115IU,26mg,29mg,2mg,0mg,9mg,1mg,44mg,25mcg,333
zesty pulled pork sandwiches,9g,3.0g,53mg,2089mg,572mg,78.6g,2.2g,25.7g,45g,465IU,5mg,106mg,2mg,0mg,10mg,0mg,32mg,42mcg,504
zucchini bites,9.4g,4.0g,64mg,253mg,167mg,7.7g,0.9g,6.2g,2g,1222IU,6mg,112mg,1mg,0mg,2mg,0mg,13mg,30mcg,138
zucchini lasagna with beef and sausage,20.5g,10.0g,112mg,1711mg,1067mg,23.4g,4.3g,28.9g,14g,2000IU,38mg,540mg,4mg,0mg,9mg,1mg,67mg,61mcg,386


In [68]:
#check csv was saved correctly! 
allrecipes_nutrdb = pd.read_csv('allrecipes_nutrdb_all_3000to4000.csv')
print(len(allrecipes_nutrdb))
allrecipes_nutrdb.tail()

911


,Unnamed: 0,Total Fat,Saturated Fat,Cholesterol,Sodium,Potassium,Total Carbohydrates,Dietary Fiber,Protein,Sugars,Vitamin A,Vitamin C,Calcium,Iron,Thiamin,Niacin,Vitamin B6,Magnesium,Folate,Calories
906,yummy pork noodle casserole,32.9g,17.0g,142mg,1575mg,615mg,32.9g,2g,38.2g,4g,1998IU,9mg,344mg,4mg,1mg,15mg,1mg,60mg,73mcg,580
907,zesty chicken and potatoes,9.9g,3.0g,59mg,558mg,725mg,41.1g,4.4g,18.5g,7g,115IU,26mg,29mg,2mg,0mg,9mg,1mg,44mg,25mcg,333
908,zesty pulled pork sandwiches,9g,3.0g,53mg,2089mg,572mg,78.6g,2.2g,25.7g,45g,465IU,5mg,106mg,2mg,0mg,10mg,0mg,32mg,42mcg,504
909,zucchini bites,9.4g,4.0g,64mg,253mg,167mg,7.7g,0.9g,6.2g,2g,1222IU,6mg,112mg,1mg,0mg,2mg,0mg,13mg,30mcg,138
910,zucchini lasagna with beef and sausage,20.5g,10.0g,112mg,1711mg,1067mg,23.4g,4.3g,28.9g,14g,2000IU,38mg,540mg,4mg,0mg,9mg,1mg,67mg,61mcg,386
